In [2]:
from keras.models import Sequential
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import style
from sklearn import svm, datasets
from sklearn.model_selection import cross_val_score
style.use("ggplot")

Using TensorFlow backend.


Download dataset
- type :  packet
- Amount : 700 packets

In [3]:
file = pd.read_csv("/srv/Program-pre/pyKMLib/examples/1000000-2-700-r.csv")

Attribute (26 features)
- Ether_or_Dot3 : A packet is ethernet type or IEEE 802.3 type
- MAC_src : MAC address of source device
- MAC_dst : MAC address of destination device
- Ether_type: The type of ethernet packet, such as 0x0800 is Internet Protocol version 4 (IPv4) 
- LLC : A packet is logical link control (LLC) type or not
- LLC_ssap : Source SAP field represents the logical address of LLC packet
- LLC_dsap : Destination SAP field represents the logical address of LLC packet
- IP_ttl : Time to live of a packet
- IP_version : The version of IP, such as IPv4 
- TCP : A packet is TCP type 
- UDP : A packet is UDP type 
- ARP : A packet is ARP type
- ICMP : A packet is ICMP type 
- pLen : A packet size
- Status : The network situation is attacked by DDoS or not
- num_ip_pair : The number of the pair of source ip and destination ip
- num_all_packets : Total of packets arrival per a window
- ratio_ip : Number of ip source divided by number of ip destination 
- num_ip_src : Number of ip source
- num_ip_dst : Number of ip destination
- num_port_pair : The number of the pair of source port and destination port

In [4]:
file.head()

,Status,Ether_or_Dot3,MAC_src,MAC_dst,Ether_type,LLC,LLC_ssap,LLC_dsap,IP_ttl,IP_version,...,Num_all_packets,Ratio_ip,Num_ip_src,Num_ip_dst,Num_port_pair,Ratio_port,Num_port_src,Num_port_dst,Weight_ip,Weight_port
0,0,1,0.0,1.0,1.0,1.0,0.0,0.0,63,1.0,...,28802,0.041421,1193,1193,1193,0.041421,1193,11254,7.21,7.21
1,0,1,0.0,1.0,1.0,1.0,0.0,0.0,63,1.0,...,18702,0.004545,85,85,85,0.004545,85,7537,1.00,1.00
2,0,1,0.0,0.0,1.0,1.0,0.0,0.0,61,1.0,...,36787,0.000870,32,32,19,0.000516,22741,19,1.00,1.00
3,1,1,0.0,0.0,1.0,1.0,0.0,0.0,61,1.0,...,4549,0.007254,33,33,28,0.006155,2677,28,1.00,1.00
4,0,1,0.0,0.0,1.0,1.0,0.0,0.0,61,1.0,...,23307,0.007122,166,166,166,0.007122,13748,166,1.00,1.00


In [5]:
file['Status'].value_counts()

0    369
1    331
Name: Status, dtype: int64

In [7]:
x_train = file.drop('Status', axis=1)
x_train = x_train.values
y_train = file['Status'].values 

In [8]:
import time
from keras.layers import Dense, Dropout, ThresholdedReLU, Lambda
from keras.utils import CustomObjectScope
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=10)
cvscores = []

for train, test in kfold.split(x_train, y_train):
    start = time.time()

    model = Sequential()
    model.add(Dense(100, input_dim=25,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=50, activation='sigmoid'))
    model.add(Dropout(0.1))
    model.add(Dense(units=20, activation='sigmoid'))
    model.add(Dense(units=1, activation='sigmoid'))
    
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    model.fit(x_train[train], y_train[train], epochs=100, batch_size=10000)
    end = time.time()
    print "train time: "+str(end-start)

    start_r = time.time()
    y_pred = model.predict_classes(x_train[test],verbose=1)
    end_r = time.time()
    print("test time: "+str(end_r-start_r))
  
    a = mean_squared_error(y_train[test], y_pred)
    b = confusion_matrix(y_train[test], y_pred)
    print 'MES= ',a
    print 'Confusion matrix= ',b
    
    scores = model.evaluate(x_train[test], y_train[test], verbose=0)
    print model.metrics_names[1]
    print "metrics: "+str(model.metrics_names[1])+" score: "+str(scores[1]*100)
    cvscores.append(scores[1] * 100)


Epoch 1/100
629/629 [==============================] - 3s 4ms/step - loss: 0.7803 - acc: 0.4722
Epoch 2/100
629/629 [==============================] - 0s 34us/step - loss: 0.7697 - acc: 0.4722
Epoch 3/100
629/629 [==============================] - 0s 35us/step - loss: 0.7699 - acc: 0.4722
Epoch 4/100
629/629 [==============================] - 0s 35us/step - loss: 0.7589 - acc: 0.4722
Epoch 5/100
629/629 [==============================] - 0s 35us/step - loss: 0.7508 - acc: 0.4722
Epoch 6/100
629/629 [==============================] - 0s 35us/step - loss: 0.7422 - acc: 0.4722
Epoch 7/100
629/629 [==============================] - 0s 35us/step - loss: 0.7288 - acc: 0.4738
Epoch 8/100
629/629 [==============================] - 0s 35us/step - loss: 0.7208 - acc: 0.4706
Epoch 9/100
629/629 [==============================] - 0s 35us/step - loss: 0.7201 - acc: 0.4674
Epoch 10/100
629/629 [==============================] - 0s 35us/step - loss: 0.7135 - acc: 0.4690
Epoch 11/100
629/629 [========

Epoch 85/100
629/629 [==============================] - 0s 34us/step - loss: 0.6533 - acc: 0.6328
Epoch 86/100
629/629 [==============================] - 0s 35us/step - loss: 0.6541 - acc: 0.6343
Epoch 87/100
629/629 [==============================] - 0s 26us/step - loss: 0.6531 - acc: 0.6391
Epoch 88/100
629/629 [==============================] - 0s 35us/step - loss: 0.6504 - acc: 0.6455
Epoch 89/100
629/629 [==============================] - 0s 35us/step - loss: 0.6510 - acc: 0.6534
Epoch 90/100
629/629 [==============================] - 0s 35us/step - loss: 0.6503 - acc: 0.6677
Epoch 91/100
629/629 [==============================] - 0s 35us/step - loss: 0.6499 - acc: 0.6566
Epoch 92/100
629/629 [==============================] - 0s 35us/step - loss: 0.6530 - acc: 0.6598
Epoch 93/100
629/629 [==============================] - 0s 34us/step - loss: 0.6483 - acc: 0.6312
Epoch 94/100
629/629 [==============================] - 0s 35us/step - loss: 0.6510 - acc: 0.6375
Epoch 95/100
629/629

Epoch 67/100
630/630 [==============================] - 0s 35us/step - loss: 0.6703 - acc: 0.6302
Epoch 68/100
630/630 [==============================] - 0s 35us/step - loss: 0.6664 - acc: 0.6365
Epoch 69/100
630/630 [==============================] - 0s 35us/step - loss: 0.6650 - acc: 0.6492
Epoch 70/100
630/630 [==============================] - 0s 35us/step - loss: 0.6696 - acc: 0.6556
Epoch 71/100
630/630 [==============================] - 0s 35us/step - loss: 0.6665 - acc: 0.6397
Epoch 72/100
630/630 [==============================] - 0s 35us/step - loss: 0.6620 - acc: 0.6603
Epoch 73/100
630/630 [==============================] - 0s 35us/step - loss: 0.6686 - acc: 0.6111
Epoch 74/100
630/630 [==============================] - 0s 35us/step - loss: 0.6667 - acc: 0.6349
Epoch 75/100
630/630 [==============================] - 0s 34us/step - loss: 0.6629 - acc: 0.6444
Epoch 76/100
630/630 [==============================] - 0s 35us/step - loss: 0.6617 - acc: 0.6381
Epoch 77/100
630/630

Epoch 49/100
630/630 [==============================] - 0s 35us/step - loss: 0.6917 - acc: 0.5143
Epoch 50/100
630/630 [==============================] - 0s 35us/step - loss: 0.6885 - acc: 0.5317
Epoch 51/100
630/630 [==============================] - 0s 35us/step - loss: 0.6871 - acc: 0.5302
Epoch 52/100
630/630 [==============================] - 0s 35us/step - loss: 0.6852 - acc: 0.5429
Epoch 53/100
630/630 [==============================] - 0s 35us/step - loss: 0.6878 - acc: 0.5524
Epoch 54/100
630/630 [==============================] - 0s 35us/step - loss: 0.6896 - acc: 0.5444
Epoch 55/100
630/630 [==============================] - 0s 35us/step - loss: 0.6882 - acc: 0.5317
Epoch 56/100
630/630 [==============================] - 0s 35us/step - loss: 0.6900 - acc: 0.5286
Epoch 57/100
630/630 [==============================] - 0s 35us/step - loss: 0.6890 - acc: 0.5270
Epoch 58/100
630/630 [==============================] - 0s 35us/step - loss: 0.6890 - acc: 0.5317
Epoch 59/100
630/630

Epoch 31/100
630/630 [==============================] - 0s 35us/step - loss: 0.6797 - acc: 0.5952
Epoch 32/100
630/630 [==============================] - 0s 35us/step - loss: 0.6737 - acc: 0.6127
Epoch 33/100
630/630 [==============================] - 0s 35us/step - loss: 0.6770 - acc: 0.6127
Epoch 34/100
630/630 [==============================] - 0s 35us/step - loss: 0.6789 - acc: 0.5905
Epoch 35/100
630/630 [==============================] - 0s 35us/step - loss: 0.6803 - acc: 0.5841
Epoch 36/100
630/630 [==============================] - 0s 35us/step - loss: 0.6780 - acc: 0.5889
Epoch 37/100
630/630 [==============================] - 0s 35us/step - loss: 0.6777 - acc: 0.6175
Epoch 38/100
630/630 [==============================] - 0s 35us/step - loss: 0.6742 - acc: 0.5952
Epoch 39/100
630/630 [==============================] - 0s 35us/step - loss: 0.6771 - acc: 0.6238
Epoch 40/100
630/630 [==============================] - 0s 35us/step - loss: 0.6756 - acc: 0.6111
Epoch 41/100
630/630

...

Epoch 61/100
630/630 [==============================] - 0s 35us/step - loss: 0.6687 - acc: 0.6286
Epoch 62/100
630/630 [==============================] - 0s 35us/step - loss: 0.6632 - acc: 0.6365
Epoch 63/100
630/630 [==============================] - 0s 34us/step - loss: 0.6657 - acc: 0.6095
Epoch 64/100
630/630 [==============================] - 0s 35us/step - loss: 0.6565 - acc: 0.6651
Epoch 65/100
630/630 [==============================] - 0s 35us/step - loss: 0.6620 - acc: 0.6365
Epoch 66/100
630/630 [==============================] - 0s 35us/step - loss: 0.6663 - acc: 0.6032
Epoch 67/100
630/630 [==============================] - 0s 34us/step - loss: 0.6599 - acc: 0.6460
Epoch 68/100
630/630 [==============================] - 0s 35us/step - loss: 0.6563 - acc: 0.6556
Epoch 69/100
630/630 [==============================] - 0s 35us/step - loss: 0.6560 - acc: 0.6508
Epoch 70/100
630/630 [==============================] - 0s 35us/step - loss: 0.6562 - acc: 0.6413
Epoch 71/100
630/630

Epoch 43/100
630/630 [==============================] - 0s 35us/step - loss: 0.6774 - acc: 0.6048
Epoch 44/100
630/630 [==============================] - 0s 35us/step - loss: 0.6766 - acc: 0.6016
Epoch 45/100
630/630 [==============================] - 0s 34us/step - loss: 0.6764 - acc: 0.5905
Epoch 46/100
630/630 [==============================] - 0s 36us/step - loss: 0.6794 - acc: 0.6000
Epoch 47/100
630/630 [==============================] - 0s 35us/step - loss: 0.6783 - acc: 0.6143
Epoch 48/100
630/630 [==============================] - 0s 35us/step - loss: 0.6756 - acc: 0.6063
Epoch 49/100
630/630 [==============================] - 0s 35us/step - loss: 0.6751 - acc: 0.6159
Epoch 50/100
630/630 [==============================] - 0s 35us/step - loss: 0.6755 - acc: 0.5968
Epoch 51/100
630/630 [==============================] - 0s 34us/step - loss: 0.6766 - acc: 0.5921
Epoch 52/100
630/630 [==============================] - 0s 35us/step - loss: 0.6736 - acc: 0.5952
Epoch 53/100
630/630

Epoch 25/100
630/630 [==============================] - 0s 35us/step - loss: 0.6761 - acc: 0.5937
Epoch 26/100
630/630 [==============================] - 0s 35us/step - loss: 0.6775 - acc: 0.5857
Epoch 27/100
630/630 [==============================] - 0s 35us/step - loss: 0.6799 - acc: 0.5889
Epoch 28/100
630/630 [==============================] - 0s 35us/step - loss: 0.6798 - acc: 0.5841
Epoch 29/100
630/630 [==============================] - 0s 35us/step - loss: 0.6764 - acc: 0.5952
Epoch 30/100
630/630 [==============================] - 0s 35us/step - loss: 0.6783 - acc: 0.6000
Epoch 31/100
630/630 [==============================] - 0s 34us/step - loss: 0.6753 - acc: 0.6095
Epoch 32/100
630/630 [==============================] - 0s 35us/step - loss: 0.6787 - acc: 0.5889
Epoch 33/100
630/630 [==============================] - 0s 35us/step - loss: 0.6729 - acc: 0.6159
Epoch 34/100
630/630 [==============================] - 0s 35us/step - loss: 0.6726 - acc: 0.6333
Epoch 35/100
630/630

Epoch 7/100
631/631 [==============================] - 0s 35us/step - loss: 0.6907 - acc: 0.5357
Epoch 8/100
631/631 [==============================] - 0s 35us/step - loss: 0.6901 - acc: 0.5436
Epoch 9/100
631/631 [==============================] - 0s 35us/step - loss: 0.6907 - acc: 0.5357
Epoch 10/100
631/631 [==============================] - 0s 35us/step - loss: 0.6866 - acc: 0.5578
Epoch 11/100
631/631 [==============================] - 0s 35us/step - loss: 0.6877 - acc: 0.5499
Epoch 12/100
631/631 [==============================] - 0s 35us/step - loss: 0.6880 - acc: 0.5452
Epoch 13/100
631/631 [==============================] - 0s 35us/step - loss: 0.6867 - acc: 0.5436
Epoch 14/100
631/631 [==============================] - 0s 35us/step - loss: 0.6839 - acc: 0.5642
Epoch 15/100
631/631 [==============================] - 0s 35us/step - loss: 0.6815 - acc: 0.5578
Epoch 16/100
631/631 [==============================] - 0s 34us/step - loss: 0.6819 - acc: 0.5626
Epoch 17/100
631/631 [=

Epoch 91/100
631/631 [==============================] - 0s 35us/step - loss: 0.6069 - acc: 0.7116
Epoch 92/100
631/631 [==============================] - 0s 34us/step - loss: 0.6115 - acc: 0.6799
Epoch 93/100
631/631 [==============================] - 0s 35us/step - loss: 0.5997 - acc: 0.7195
Epoch 94/100
631/631 [==============================] - 0s 35us/step - loss: 0.6121 - acc: 0.6799
Epoch 95/100
631/631 [==============================] - 0s 35us/step - loss: 0.5885 - acc: 0.7448
Epoch 96/100
631/631 [==============================] - 0s 35us/step - loss: 0.6069 - acc: 0.7021
Epoch 97/100
631/631 [==============================] - 0s 34us/step - loss: 0.6051 - acc: 0.6926
Epoch 98/100
631/631 [==============================] - 0s 35us/step - loss: 0.6042 - acc: 0.6973
Epoch 99/100
631/631 [==============================] - 0s 34us/step - loss: 0.6070 - acc: 0.6767
Epoch 100/100
631/631 [==============================] - 0s 36us/step - loss: 0.6117 - acc: 0.6672
train time: 3.45550

1000000 with version packets dataset

In [9]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

68.30% (+/- 6.11%)


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 100)               2600      
_________________________________________________________________
dropout_19 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_20 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 21        
Total params: 8,691
Trainable params: 8,691
Non-trainable params: 0
_________________________________________________________________


Save model

In [10]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model-maxp-dff.json","w") as json_file:
    json_file.write(model_json)
model.save_weights("model-maxp-dff.h5")